In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import cv2
import scipy.io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# File paths
mat_file_path = "/Users/keshavsaraogi/data/pose/joints.mat"
image_folder = "/Users/keshavsaraogi/data/pose/images/"

In [ ]:
# Load joint data
data = scipy.io.loadmat(mat_file_path)
joints = data["joints"]  # Shape: (3, 14, 2000)

In [ ]:
# Preprocess images
IMG_SIZE = 128  # Resize images to 128x128
num_images = joints.shape[2]

In [ ]:
X = []
y = []

for img_index in range(num_images):
    img_path = os.path.join(image_folder, f"im{img_index+1:04d}.jpg")
    
    # Read image
    img = cv2.imread(img_path)
    if img is None:
        continue  # Skip missing images

    # Resize and normalize image
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize

    # Get keypoints & normalize to [0,1]
    keypoints = joints[:, :, img_index][:2, :]  # Extract (x, y)
    keypoints = keypoints.flatten()  # Convert to (28,) vector

    X.append(img)
    y.append(keypoints)


In [ ]:
# Convert to NumPy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [ ]:
# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

In [ ]:
# Build Simple CNN Model
def create_simple_cnn():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(28, activation="linear")  # 14 keypoints (x,y) -> 28 values
    ])
    
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model


In [ ]:
# Create and Train Model
model = create_simple_cnn()
model.summary()

In [ ]:
# Train
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Save model
model.save("simple_cnn_pose.keras")
print("Training complete. Model saved as 'simple_cnn_pose.keras'.")

## CUSTOM MADE CNN 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def create_custom_cnn(input_shape=(128, 128, 3), num_keypoints=28):
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        
        # Block 4
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        # Flatten layer
        Flatten(),
        
        # Print shape after Flattening
        Dense(1024, activation='relu'),  # Fixing dense input shape mismatch
        Dropout(0.5),
        
        # Output layer (14 keypoints * 2 coordinates each)
        Dense(num_keypoints, activation='linear')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

    return model

# Initialize model
custom_cnn_model = create_custom_cnn()

# Print summary to verify
custom_cnn_model.summary()


In [ ]:
# Initialize model
custom_cnn_model = create_custom_cnn()

# Print summary
custom_cnn_model.summary()

In [ ]:
# Train the custom model
history = custom_cnn_model.fit(
    X_train, y_train, 
    epochs=25,  # Increase epochs for better performance
    batch_size=32, 
    validation_data=(X_test, y_test)
)

# Save the model
custom_cnn_model.save("custom_cnn_pose.h5")
print("Training complete. Model saved as 'custom_cnn_pose.h5'.")